In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import lit
from delta.tables import DeltaTable
from pyspark.sql.functions import col, row_number

In [0]:
# Step 1: Read CSV
oct_df = spark.read.csv(
    "/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv",
    header=True,
    inferSchema=True
)

In [0]:
# Step 2: Set Delta table path
delta_path = "/dbfs/tmp/ecommerce_events"

In [0]:
# Step 3: Save as Delta table (overwrite if exists)
oct_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("ecommerce_events")

In [0]:
# Step 5: Load Delta table
delta_table = DeltaTable.forName(spark, "ecommerce_events")

In [0]:
# Step 6: Deduplicate source to avoid multiple source row errors
window_spec = Window.partitionBy(
    "user_session",
    "event_time",
    "product_id",
    "event_type"
).orderBy(col("event_time").desc())

updates_df = (
    oct_df
    .withColumn("rn", row_number().over(window_spec))
    .filter(col("rn") == 1)
    .drop("rn")
)

In [0]:
# Step 7: Check count before merge
before_count = spark.read.format("delta").table("ecommerce_events").count()
print("Before merge count of rows:", before_count)

Before merge count of rows: 42448764


In [0]:
# Step 8: Perform MERGE
delta_table.alias("t").merge(
    updates_df.alias("s"),
    """
    t.user_session = s.user_session AND
    t.event_time = s.event_time AND
    t.product_id = s.product_id AND
    t.event_type = s.event_type
    """
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
# Step 9: Check count after merge
after_count = spark.read.format("delta").table("ecommerce_events").count()
print("After merge count of rows:", after_count)

After merge count of rows: 42448766


In [0]:
from delta.tables import DeltaTable

# Load table
delta_table = DeltaTable.forPath(spark, delta_path)

In [0]:
# a) Show Delta history
spark.sql("DESCRIBE HISTORY ecommerce_events").show(truncate=False)

+-------+-------------------+--------------+------------------------+---------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+------------------+------------------------+-----------+-----------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
# b) Query an old version by version number
version_0_df = spark.read.format("delta") \
    .option("versionAsOf", 0) \
    .table("ecommerce_events")

print("Number of rows in version 0:", version_0_df.count())

Number of rows in version 0: 42448764


In [0]:
# c) Query as of a timestamp
from datetime import datetime

yesterday_df = spark.read.format("delta") \
    .option("timestampAsOf", "2026-01-13 16:02:24") \
    .table("ecommerce_events")

print("Number of rows as of timestamp:", yesterday_df.count())

Number of rows as of timestamp: 42448764


In [0]:
# Optimize Delta table with ZORDER by columns often queried
spark.sql("OPTIMIZE ecommerce_events ZORDER BY (event_type, user_id)")

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

In [0]:
# retention: 7 days (168 hours)
delta_table.vacuum(retentionHours=168)

---------------------------------------------------------------------------
UnsupportedOperationException             Traceback (most recent call last)
File <command-8375019326151658>, line 2
      1 # retention: 7 days (168 hours)
----> 2 delta_table.vacuum(retentionHours=168)

File /databricks/python/lib/python3.12/site-packages/delta/connect/tables.py:177, in DeltaTable.vacuum(self, retentionHours)
    175 def vacuum(self, retentionHours: Optional[float] = None) -> DataFrame:
    176     command = Vacuum(self._to_proto(), retentionHours).command(session=self._spark.client)
--> 177     self._spark.client.execute_command(command)
    178     return None

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/client/core.py:1556, in SparkConnectClient.execute_command(self, command, observations, extra_request_metadata)
   1554     req.user_context.user_id = self._user_id
   1555 req.plan.command.CopyFrom(command)
-> 1556 data, _, metrics, observed_metrics, properties 

In [0]:
# retention: 7 days (168 hours)
delta_table = DeltaTable.forName(spark, "ecommerce_events")
delta_table.vacuum(retentionHours=168)